In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

trainloader = unpickle('../datasets/cifar-10/data_batch_1')

testloader = unpickle('../datasets/cifar-10/test_batch')

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn


class Net(nn.Module):
	def __init__(self):
		super(Net, self).__init__()
		self.conv1 = nn.Conv2d(3, 6, 5)
		self.pool = nn.MaxPool2d(2, 2)
		self.conv2 = nn.Conv2d(6, 16, 5)
		self.fc1 = nn.Linear(16 * 5 * 5, 120)
		self.fc2 = nn.Linear(120, 84)
		self.fc3 = nn.Linear(84, 10)
	
	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		x = x.view(-1, 16 * 5 * 5)
		x = F.relu(self.fc1(x))
		x = F.relu(self.fc2(x))
		x = self.fc3(x)
		return x

net = Net()
net.cuda()
cudnn.benchmark=True

In [38]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

train_labels = trainloader[b'labels']
train_inputs = trainloader[b'data']

print(train_labels[0])

for epoch in range(2):  # 这里只迭代2个epoch，实际应该进行更多次训练 
    running_loss = 0.0
    for i in range(len(train_labels)):
        # 得到输入
        inputs, label = train_inputs[i], train_labels[i]

        inputs = inputs / 128.0 - 1.0
        inputs = torch.from_numpy(inputs)
        inputs = inputs.view(1,3,32,32).type(torch.FloatTensor).cuda()
        
        labels = torch.zeros(10)
        labels[label] = 1
        labels = labels.type(torch.LongTensor).cuda()
        print(labels)

        # 梯度清零 
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)[0]
        print(outputs.data)
        loss = criterion(outputs.data, labels)
        loss.backward()
        optimizer.step()

        # 定义统计信息
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

6
tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0], device='cuda:0')
tensor([-0.1264, -0.0080,  0.0908, -0.0436, -0.0360,  0.0789,  0.0657, -0.0384,
        -0.0627,  0.0025], device='cuda:0')


RuntimeError: Dimension out of range (expected to be in range of [-1, 0], but got 1)